[View in Colaboratory](https://colab.research.google.com/github/ami432/INKERS/blob/master/Bharatkumar_Batch3_Assignment4B_87.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size =64
num_classes = 10
epochs = 40
l = 10
num_filter = 48
compression = 0.5
dropout_rate = 0.3

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 48
dropout_rate = 0.2
l = 10
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 48)   1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 48)   192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 48)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# checkpoint
filepath="CIFAR10-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='auto',period=1)
callbacks_list = [checkpoint]


In [13]:
# Fit the model
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
43776/50000 [=========================>....] - ETA: 46s - loss: 1.4345 - acc: 0.4770

50000/50000 [==============================] - 396s 8ms/step - loss: 1.3966 - acc: 0.4913 - val_loss: 1.3281 - val_acc: 0.5430

Epoch 00001: saving model to CIFAR10-weights-improvement-01-0.54.hdf5
Epoch 2/40
19840/50000 [==========>...................] - ETA: 3:39 - loss: 1.0112 - acc: 0.6364

50000/50000 [==============================] - 386s 8ms/step - loss: 0.9506 - acc: 0.6596 - val_loss: 1.2628 - val_acc: 0.6100

Epoch 00002: saving model to CIFAR10-weights-improvement-02-0.61.hdf5
Epoch 3/40
12928/50000 [======>.......................] - ETA: 4:29 - loss: 0.7971 - acc: 0.7168

50000/50000 [==============================] - 385s 8ms/step - loss: 0.7729 - acc: 0.7257 - val_loss: 1.2338 - val_acc: 0.6327

Epoch 00003: saving model to CIFAR10-weights-improvement-03-0.63.hdf5
Epoch 4/40
10944/50000 [=====>........................] - ETA: 4:43 - loss: 0.6814 - acc: 0.7608

50000/50000 [==============================] - 385s 8ms/step - loss: 0.6640 - acc: 0.7689 - val_loss: 0.7273 - val_acc: 0.7626

Epoch 00004: saving model to CIFAR10-weights-improvement-04-0.76.hdf5
Epoch 5/40
10368/50000 [=====>........................] - ETA: 4:48 - loss: 0.6094 - acc: 0.7845

...

50000/50000 [==============================] - 385s 8ms/step - loss: 0.1099 - acc: 0.9596 - val_loss: 0.6261 - val_acc: 0.8598

Epoch 00036: saving model to CIFAR10-weights-improvement-36-0.86.hdf5
Epoch 37/40
10048/50000 [=====>........................] - ETA: 4:50 - loss: 0.0997 - acc: 0.9647

50000/50000 [==============================] - 385s 8ms/step - loss: 0.1090 - acc: 0.9609 - val_loss: 0.5594 - val_acc: 0.8714

Epoch 00037: saving model to CIFAR10-weights-improvement-37-0.87.hdf5
Epoch 38/40
10048/50000 [=====>........................] - ETA: 4:49 - loss: 0.1009 - acc: 0.9635

50000/50000 [==============================] - 385s 8ms/step - loss: 0.1030 - acc: 0.9632 - val_loss: 0.5878 - val_acc: 0.8642

Epoch 00038: saving model to CIFAR10-weights-improvement-38-0.86.hdf5
Epoch 39/40
10048/50000 [=====>........................] - ETA: 4:50 - loss: 0.0831 - acc: 0.9712

50000/50000 [==============================] - 385s 8ms/step - loss: 0.1005 - acc: 0.9645 - val_loss: 0.9451 - val_acc: 0.7973

Epoch 00039: saving model to CIFAR10-weights-improvement-39-0.80.hdf5
Epoch 40/40
10048/50000 [=====>........................] - ETA: 4:50 - loss: 0.0847 - acc: 0.9699

50000/50000 [==============================] - 385s 8ms/step - loss: 0.0966 - acc: 0.9653 - val_loss: 0.5731 - val_acc: 0.8758

Epoch 00040: saving model to CIFAR10-weights-improvement-40-0.88.hdf5


In [14]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 27s 3ms/step
Test loss: 0.5730647237718105
Test accuracy: 0.8758


In [15]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')